In [ ]:
##! pip install numpy # Numpy is required by surprise
##! pip install scikit-surprise

# Patio, lawn and garden ratings

This dataset was downloaded from Amazon dataset that is publicly available at (http://jmcauley.ucsd.edu/data/amazon/). This dataset contains user reviews (numerical rating and textual comment) towards amazon products on patio, lawn and garden products category.


# Read Data

In [1]:
import pandas as pd

In [2]:
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

# df = getDF('Patio_Lawn_and_Garden_5.jason')
df = getDF('C:/Users/magpanta/Documents/IE University/IE Global Master on Big Data/Classes/21. Recommendations engines/Assignment/reviews_Patio_Lawn_and_Garden_5.json.gz')

In [3]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1JZFGZEZVWQPY,B00002N674,"Carter H ""1amazonreviewer@gmail . com""","[4, 4]",Good USA company that stands behind their prod...,4.0,Great Hoses,1308614400,"06 21, 2011"
1,A32JCI4AK2JTTG,B00002N674,"Darryl Bennett ""Fuzzy342""","[0, 0]",This is a high quality 8 ply hose. I have had ...,5.0,Gilmour 10-58050 8-ply Flexogen Hose 5/8-Inch ...,1402272000,"06 9, 2014"
2,A3N0P5AAMP6XD2,B00002N674,H B,"[2, 3]",It's probably one of the best hoses I've ever ...,4.0,Very satisfied!,1336176000,"05 5, 2012"
3,A2QK7UNJ857YG,B00002N674,Jason,"[0, 0]",I probably should have bought something a bit ...,5.0,Very high quality,1373846400,"07 15, 2013"
4,AS0CYBAN6EM06,B00002N674,jimmy,"[1, 1]",I bought three of these 5/8-inch Flexogen hose...,5.0,Good Hoses,1375660800,"08 5, 2013"
...,...,...,...,...,...,...,...,...,...
13267,AT53ZTTO707MB,B00KS0F4FI,I Do The Speed Limit,"[1, 2]",Simple. Perfect. Plenty big enough. Durable...,5.0,Great pair of claws,1403827200,"06 27, 2014"
13268,AYB4ELCS5AM8P,B00KS0F4FI,"John B. Goode ""JBG""","[0, 0]",These claws are fantastic. They are made of ha...,5.0,Sharp...,1405123200,"07 12, 2014"
13269,AZMY6E8B52L2T,B00KS0F4FI,"JP ""J.P.""","[1, 3]",I really like theseOuddy Heat Resistant Meat C...,5.0,Really Helpful...,1405123200,"07 12, 2014"
13270,AEC90GPFKLAAW,B00KS0F4FI,"Lisa Kearns ""Lisa Kearns""","[0, 0]",I make pulled pork in the crock pot pretty oft...,5.0,Shreds meat perfectly!,1405468800,"07 16, 2014"


Visualizing the dataset to ensure that everything looks good

In [4]:
print('Dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::20000, :])

Dataset shape: (13272, 9)
-Dataset examples-
       reviewerID        asin                            reviewerName helpful  \
0  A1JZFGZEZVWQPY  B00002N674  Carter H "1amazonreviewer@gmail . com"  [4, 4]   

                                          reviewText  overall      summary  \
0  Good USA company that stands behind their prod...      4.0  Great Hoses   

   unixReviewTime   reviewTime  
0      1308614400  06 21, 2011  


# Analyze the data


### Ratings distribution

Here we are analyzing the distribution of ratings to understand among other things:

- The rating scale: is it 0-5, 1-5, 1-10....
- The distribution of the ratings: are the users biased to high or low rates?
- Are the users using the entire rating scale? e.g.: sometimes people performs binary ratings even if you provide them with a more granular scale (5 good, 1 bad and nothing in between)


In [9]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# Count the number of times each rating appears in the dataset
data = df['overall'].value_counts().sort_index(ascending=False)

# Create the histogram
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution Of {} patio, lawn and garden ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

### Numbers of ratings per patio, lawn and garden

The following analysis checks how many ratings (interactions) each category has.

In [10]:
# Number of ratings per garden equipment
data = df.groupby('asin')['overall'].count()

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per garden',
                   xaxis = dict(title = 'Number of Ratings Per garden'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Most of the garden category received less than 10 ratings, and very few have many ratings. This is very typical (and in fact one of the main problems) of Recommender Systems.



### Number of ratings per user

Below provides how many ratings we have per user. Because of how the dataset was created, the minimum number of ratings per user in this dataset is 20, which alleviates the long-tail problem (i.e., when most of your users has very few or 0 ratings). 

In [12]:
# Number of ratings per reviewer
data = df.groupby('reviewerID')['overall'].count()

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0, size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User',
                   xaxis = dict(title = 'Ratings Per reviewerID'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

# Managing data with Surprise

Below the dataset was converted to the format required by the Surprise library. 
To load a data set from the above pandas data frame, we will use the *load_from_df()* method, we will also need a Reader object, and the rating_scale parameter must be specified. The data frame has three columns, corresponding to the user id (reviewerID), the item id (asin), and the ratings (overall) in this order. Each row thus corresponds to a given rating.

In [13]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split

# df2 = df.iloc[:30000, :]
# df2 = df2.groupby('reviewerID').filter(lambda x: len(x) >= 5)
data_train, data_test = train_test_split(df, test_size=0.20, random_state=42, stratify=df['reviewerID'])

In [16]:
from surprise import Dataset
from surprise import Reader
reader = Reader(rating_scale=(1, 5))
data_train_cf = Dataset.load_from_df(data_train[['reviewerID', 'asin', 'overall']], reader)
data_test_cf = Dataset.load_from_df(data_test[['reviewerID', 'asin', 'overall']], reader)

# First Try: Neighbourhood-based Collaborative Filtering

Here we are using one of the simplest recommendation methodologies (Neighbourhood-based CF). This technique is pretty simple and fast, but it provides accurate results for many scenarios.

To specify the parameters of the execution, you simply have to configure the function by passing a dictionary as an argument to the recommender function. The dictionary should have the required keys, such as the following:

- **name** contains the similarity metric to use. Options are cosine, msd, pearson, or pearson_baseline. The default is msd.
- **user_based** is a boolean that tells whether the approach will be user-based or item-based. The default is True, which means the user-based approach will be used.
- **min_support** is the minimum number of common items needed between users to consider them for similarity. For the item-based approach, this corresponds to the minimum number of common users for two items.
    
In particular, I will use the cosine distance as similarity metric for finding the neighbours, using the item-based approach

In [21]:
from surprise import KNNBaseline

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
knn = KNNBaseline(sim_options=sim_options)

In [22]:
from surprise.model_selection import cross_validate

results = cross_validate(knn, data, measures=['RMSE'], cv=3, verbose=True)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0786  1.0839  1.0480  1.0702  0.0158  
Fit time          0.05    0.04    0.05    0.05    0.01    
Test time         0.09    0.06    0.05    0.07    0.02    


The RMSE is about 1.08 (i.e., the average error between the predicted and the actual ratings). 

In order to actually understand the value of these results,we need to compare them to a very simple baseline approach implementing whatever logic or business rule you consider valid. For example: randomly recommend from the 10 most common books. 

In [23]:
from surprise import NormalPredictor

cross_validate(NormalPredictor(), data, measures=['RMSE'], cv=3, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.4273  1.3910  1.4055  1.4079  0.0149  
Fit time          0.01    0.01    0.02    0.01    0.00    
Test time         0.02    0.00    0.02    0.01    0.01    


{'test_rmse': array([1.42727926, 1.39095988, 1.40545519]),
 'fit_time': (0.01196908950805664, 0.009009361267089844, 0.01562952995300293),
 'test_time': (0.021945953369140625,
  0.003842592239379883,
  0.015622854232788086)}

RMSE is smallerwhat tells us that:
- KNN is actually learning from the training data
- 1.4 is the lower threshold for any recommender system in this dataset

## Tuning the Algorithm Parameters

As we explained, you can provide a dictionary with the desired configuration (i.e., similarity metric, user vs item based,...) to your recommendation algorithm. Ideally, you would like to test different configurations and select the one that best performs for your dataset and recommendation problem.

To that end, Surprise provides a GridSearchCV class analogous to GridSearchCV from scikit-learn.
With a dict of all parameters, GridSearchCV tries all the combinations of parameters and reports the best parameters for any accuracy measure

For example, you can check which similarity metric works best for your data in memory-based approaches:

In [24]:
from surprise import KNNBaseline
from surprise.model_selection import GridSearchCV

sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

The best configuration corresponds to a item-based configuration using Mean Square distance with a min support equals to 5, which is able to slightly reduce the RMSE to 1.0057.


In [25]:
from surprise import AlgoBase
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import PredictionImpossible

class GroupAlgorithm(AlgoBase):

    def __init__(self, sim_options={}, bsl_options={}):

        AlgoBase.__init__(self, sim_options=sim_options, bsl_options=bsl_options)

    def fit(self, trainset):

        AlgoBase.fit(self, trainset)

        # Compute baselines and similarities
        self.bu, self.bi = self.compute_baselines()
        self.sim = self.compute_similarities()

        return self

    def estimate(self, u, i):

        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unknown.')

        # Compute similarities between u and v, where v describes all other users that have also rated item i.
        neighbors = [(v, self.sim[u, v]) for (v, r) in self.trainset.ir[i]]
        # Sort these neighbors by similarity
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

        print('The 3 nearest neighbors of user', str(u), 'are:')
        for v, sim_uv in neighbors[:3]:
            print('user {0:} with sim {1:1.2f}'.format(v, sim_uv))

        # Return the baseline estimate
        bsl = self.trainset.global_mean + self.bu[u] + self.bi[i]
        return bsl


# Second Try: Matrix Factorization

Using SVD++ which usually offers state of the art results for many recommendation tasks

In [26]:
from surprise import SVDpp

# We'll use the famous SVD algorithm.
svd = SVDpp()

results = cross_validate(svd, data, measures=['RMSE'], cv=3, verbose=False)

# Third Try: Benchmarking

Here we are going to experiment with different algorithms to check which one of them offers the best results.



In [27]:
from surprise import SVD
from surprise import BaselineOnly
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering

benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), BaselineOnly(), CoClustering()]:
    
    print("Testing {}".format(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

Testing <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x00000295A9A5C790>
Testing <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x00000295A9A5C940>
Testing <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x00000295A9A5C910>
Testing <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x00000295A9A5C8B0>
Testing <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x00000295AA08F040>
Testing <surprise.prediction_algorithms.knns.KNNBaseline object at 0x00000295AA08F250>
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Testing <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x00000295AA08F370>
Estimating biases using als...

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.993401,1.174140,0.062149
SVD,0.994504,0.359375,0.020254
BaselineOnly,0.996672,0.015642,0.004011
KNNBaseline,1.074627,0.054016,0.109916
CoClustering,1.093463,0.290022,0.012911
SlopeOne,1.180429,0.029194,0.021796
NMF,1.205355,0.459217,0.017703
NormalPredictor,1.386840,0.010172,0.016997


As expected the best results are offered by matrix factorization, particularly SVD++. However, it took longer time to train (especially compared to SVD). 

Neighbourhood-based methods (KNN) also works pretty well for this dataset. My recommendation is that you always should try it.

Surprisingly, the BaselineOnly approach is offering very competitive results in spite of its simplicity. 

# Interpretation

In this final section I would like to provide you with some examples about how to interpret the results and execution of your recommendation system.

This interpretation is very useful to understand the operation of the recommendation system and to be able to improve their results, as well as to explain the recommendation to relevant stakeholders.

The code is a little bit tricky and it will be different if you use a different library (you should research about how to replicate this analysis with your preferred library), so don't get discouraged if you get lost in the Python Code and focus on iterpreting and understanding the results and to capture the idea.

## Get Neigbours

Here we would like to inspect the neighbourhoods in which the recommendations are generated. We took an item and analyzed their neighbourhood to check if it makes sense (i.e., if it includes items or users similar to the one that you are analyzing).

The following code gets the 10 closest neighbours to a given garden item.

In [37]:
# Execute KNN
sim_options = {'name': 'pearson_baseline', 'user_based': False}
knn = KNNBaseline(sim_options=sim_options)
knn.fit(data.build_full_trainset())

# Target item to analyze its neighbourhood
garden_name = 'B00002N674'

# Get the closes neighbourds
neighbors = knn.get_neighbors(knn.trainset.to_inner_iid(garden_name), k=10)
# Translate the internal ids used in the algorithm to the movie names
neighbors = (knn.trainset.to_raw_iid(asin) for asin in neighbors)

print()
print('The 10 nearest neighbors of {} are:\n'.format(garden_name))
for asin in neighbors:
    print("\t",asin)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

The 10 nearest neighbors of B00002N674 are:

	 B000WEOQV8
	 B00002N67P
	 B00002N67Q
	 B00002N68C
	 B00002N6AN
	 B00002N8K3
	 B00002N8NR
	 B00004DTNH
	 B00004R9TK
	 B00004R9TL


## Check Factorized Matrices

Here we analyze the resulting matrices of SVD to understand what the algorithm is doing.

In particular, we analyze 2 things:
- **Bias**: Bias terms try to consider the tendencies in the ratings to be biased to a particular aspect. 
- **Factors**: These are the underlying factors that are driving the ratings that we expect the matrix factorization algorithm to learn from the rating matrix.

### Analyze BIAS

Herev we analyze the bias using Surprise.

#### Item Bias

Here we analyzed the bias related to the items to understand if there is any bias in their ratings. 

We list here the patio, lawn and garden with a large negative (worst reviews) and positive (reviews) bias.

In [40]:
garden_bias = [(b, data.build_full_trainset().to_raw_iid(i)) for i, b in enumerate(svd.bi)]
print("Worst garden ever:")
sorted(garden_bias, key=lambda x: x[0])[:15]

Worst garden ever:


[(-1.312142188559179, 'B0008JGSD6'),
 (-1.2701316633878343, 'B001JJCW0M'),
 (-1.12292198608524, 'B001RCTX4E'),
 (-1.0596652088646055, 'B00004R9TL'),
 (-1.0596354345705263, 'B000S61YO2'),
 (-1.043449766483747, 'B00004U9VV'),
 (-1.0102852829706717, 'B002NKYWJA'),
 (-0.9693732857966942, 'B000HCLLMM'),
 (-0.9543767573497065, 'B002WJIQGC'),
 (-0.9264143024407903, 'B00004RAMT'),
 (-0.9232840052548662, 'B000HHQ2PI'),
 (-0.9176397975013463, 'B000P0DK1Q'),
 (-0.9142285858708707, 'B000F0BVYE'),
 (-0.9131524527073478, 'B001B3PI1G'),
 (-0.8663561085386531, 'B003TQM9XS')]

In [39]:
print("Worst garden ever:")
sorted(garden_bias, key=lambda x: -x[0])[:15]

Worst garden ever:


[(0.6827760036624956, 'B000Y1BGN0'),
 (0.5796522968621854, 'B000H5SD5C'),
 (0.5696715176334085, 'B00004SD6U'),
 (0.5531347453749209, 'B000BQPW6A'),
 (0.5526923930982236, 'B000I1BX7U'),
 (0.5516166010243709, 'B008RGCA9I'),
 (0.5482490207309935, 'B000OV8OTY'),
 (0.5471803030238303, 'B00004TBKM'),
 (0.5450539552825246, 'B000EDVTLW'),
 (0.5323464812186883, 'B00602J3V4'),
 (0.5300701226119071, 'B000WEMG24'),
 (0.5201381795766494, 'B0007OPJM8'),
 (0.5180569498357442, 'B00009PR1E'),
 (0.5097399953978055, 'B000UGT94C'),
 (0.5061619217859611, 'B001TRELMQ')]

#### User Bias

Here we can perform the same bias analysis over the users(reviewersID). The idea is the following: there are people that consistently rate garden very well or very badly, no matter the garden quality.

The following code prints the user with the most negative bias

In [46]:
user_bias = [(b, data.build_full_trainset().to_raw_uid(i)) for i, b in enumerate(svd.bu)]
sorted(user_bias, key=lambda x: x[0])[-1]

(0.7720203480910044, 'A2I2KPNJDQ9SL0')

In [47]:
df[df.reviewerID == 'A2I2KPNJDQ9SL0']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
200,A2I2KPNJDQ9SL0,B00004RA4G,XiMiX,"[0, 0]","This works well, but you cannot tell if it jus...",4.0,Works good,1267488000,"03 2, 2010"
3822,A2I2KPNJDQ9SL0,B000LNWIVU,XiMiX,"[0, 0]",Works amazingly well in catching flies. I buy ...,4.0,Catches flies like crazy,1287187200,"10 16, 2010"
4612,A2I2KPNJDQ9SL0,B000W72GBC,XiMiX,"[0, 0]",I find most CARB-compliant gas cans to be diff...,5.0,Easy to use CARB gas can,1360454400,"02 10, 2013"
5601,A2I2KPNJDQ9SL0,B0018U0A3S,XiMiX,"[1, 2]","It's just a chain oil, and there has not been ...",5.0,OK,1373673600,"07 13, 2013"
8898,A2I2KPNJDQ9SL0,B005968T1M,XiMiX,"[0, 0]","I use it around the house, hidden behind furni...",5.0,Works well,1354838400,"12 7, 2012"
11156,A2I2KPNJDQ9SL0,B00AW72V4E,XiMiX,"[0, 1]",The 40V 4AH battery is not the same thing as A...,4.0,"All the benefit of battery power, except the w...",1371600000,"06 19, 2013"
12135,A2I2KPNJDQ9SL0,B00DRBBRVU,XiMiX,"[18, 20]","As others have stated, it is a very good blowe...",3.0,Very good blower. OK for vacuum,1382227200,"10 20, 2013"


In contrast, if we analyze the user with the most positive bias, we will see that they scores are pretty high.

In [48]:
sorted(user_bias, key=lambda x: x[0])[0]

(-1.5390178176832687, 'A1NKUPJZFRD0HB')

In [50]:
df[df.reviewerID == 'A1NKUPJZFRD0HB']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
455,A1NKUPJZFRD0HB,B00004SD76,Cimmerian,"[0, 0]","I have to say,I was a little worried that eith...",5.0,Had these for a while now!....,1357084800,"01 2, 2013"
1291,A1NKUPJZFRD0HB,B00062KQ42,Cimmerian,"[0, 0]",Most people have already said what needs to be...,5.0,Awesome product,1390435200,"01 23, 2014"
2095,A1NKUPJZFRD0HB,B000BNKWZY,Cimmerian,"[0, 0]",For Hydro systems as well as straight nutrient...,5.0,Fantastic product!!!!,1346716800,"09 4, 2012"
2587,A1NKUPJZFRD0HB,B000E7MTT4,Cimmerian,"[3, 3]",The only small thing Id like to mention is tha...,5.0,Perfect,1363651200,"03 19, 2013"
6482,A1NKUPJZFRD0HB,B001UV6P9I,Cimmerian,"[4, 4]",I looked at the Lumens output for my needs and...,5.0,Nice Light!!!!,1346716800,"09 4, 2012"
6778,A1NKUPJZFRD0HB,B002JUOWB2,Cimmerian,"[4, 5]",I have two of these and also one of the same t...,5.0,Seems nice so far.,1363651200,"03 19, 2013"
8906,A1NKUPJZFRD0HB,B005CY100I,Cimmerian,"[0, 0]","I own two sets of these hangers ,and i gotta s...",5.0,These are really REALLY nice!!,1350518400,"10 18, 2012"
10291,A1NKUPJZFRD0HB,B008NYF8S4,Cimmerian,"[0, 0]",I didnt know much about carbon filters and hav...,5.0,seems to work well,1400284800,"05 17, 2014"
